In [308]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [309]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 5
EVENT_NAME = 'CUSUM_0.001'
LABEL_NAME = "FH"

START_DATE = "20210101"
END_DATE = "20241231"

RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "interm/events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_FEATURES.pkl"
EVENT_FILE_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}.pkl"

In [310]:
DIRECTION_LABEL_DIR = BASE_DIR / "labels/direction_labels"
DIRECTION_LABEL_DIR.mkdir(parents=True, exist_ok=True)
DIRECTION_LABEL_FILE_PATH = DIRECTION_LABEL_DIR / f"{RESAMPLED_NAME}-{EVENT_NAME}-{LABEL_NAME}.pkl"

In [311]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)
t_events = pd.read_pickle(EVENT_FILE_PATH)

CPU times: user 1.15 ms, sys: 20.8 ms, total: 21.9 ms
Wall time: 20.4 ms


In [312]:
df.loc[t_events].head()

,open,high,low,close,volume,spread,close_pct_return,close_return,close_log_return,close_fd_return,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-07 05:45:00,103.1925,103.2260,103.1890,103.2135,7.047600e+11,0.002410,0.000199,0.0205,0.000199,1.954618,...,7,1,0.965926,0.258819,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025
2021-01-07 07:05:00,103.2515,103.3085,103.2500,103.2995,8.774100e+11,0.002593,0.000460,0.0475,0.000460,1.993861,...,7,1,0.965926,-0.258819,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025
2021-01-07 08:05:00,103.3590,103.3945,103.3560,103.3795,1.817860e+12,0.002563,0.000189,0.0195,0.000189,1.972264,...,7,1,0.866025,-0.500000,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025
2021-01-07 09:10:00,103.4220,103.4730,103.4160,103.4660,1.227620e+12,0.002118,0.000421,0.0435,0.000421,2.002900,...,7,1,0.707107,-0.707107,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025
2021-01-07 09:50:00,103.5100,103.5775,103.5085,103.5755,2.519750e+12,0.001983,0.000623,0.0645,0.000623,2.044747,...,7,1,0.707107,-0.707107,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025


## Calculate Target Volatility (trgt)
This will be used for the calculation of dynamic TP/SL


In [313]:
import pandas_ta as ta

In [314]:
# 預計持單 1 小時 (60 minutes)
# 計算 1 小時內價格可能到哪裡
# 計算 1 小時 normalized atr
atr12 = df.ta.natr(length=12)

In [315]:
atr12

timestamp
2021-01-07 03:25:00         NaN
2021-01-07 03:30:00         NaN
2021-01-07 03:35:00         NaN
2021-01-07 03:40:00         NaN
2021-01-07 03:45:00         NaN
                         ...   
2024-12-30 23:35:00    0.024607
2024-12-30 23:40:00    0.024118
2024-12-30 23:45:00    0.025192
2024-12-30 23:50:00    0.024895
2024-12-30 23:55:00    0.024015
Name: NATR_12, Length: 298164, dtype: float64

In [316]:
trgt = atr12.reindex(t_events, method='ffill') / 100 # rate

In [317]:
(trgt).describe()

count    24493.000000
mean         0.000640
std          0.000423
min          0.000062
25%          0.000377
50%          0.000542
75%          0.000770
max          0.004740
Name: NATR_12, dtype: float64

## Apply Triple Barrier
We use daily volatility to calculate TP/SL, and we set our T1 as 1day

In [318]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd

def applyPtSlOnT1(close,events,ptSl,molecule):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    
    pt = ptSl[0] * events_['trgt'] if ptSl[0] > 0 else pd.Series(index=events.index)
    sl = -ptSl[1] * events_['trgt'] if ptSl[1] > 0 else pd.Series(index=events.index)

        
    for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
        df0=close[loc:t1] # path prices
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns (rate)
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking
        
    return out


def parallel_apply(func, items, num_threads=4, **kwargs):
    
    def worker(molecule):
        return func(molecule=molecule, **kwargs)

    chunks = np.array_split(items, num_threads)
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(worker, chunks))

    return pd.concat(results).sort_index()


def getEvents(close, tEvents, ptSl, trgt, minTrgt, numThreads=4, t1=False, side=None):
    # Step 1: Filter targets
    trgt = trgt.loc[tEvents]
    trgt = trgt[trgt > minTrgt]

    # Step 2: Set vertical barrier (t1)
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=tEvents)

    # Step 3: Build events DataFrame
    if side is None:
        side_, ptSl_ = pd.Series(1., index=trgt.index), [ptSl[0], ptSl[0]]
    else:
        side_, ptSl_ = side.loc[trgt.index], ptSl[:2]

    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1).dropna(subset=['trgt'])

    # Step 4: Apply barriers in parallel
    df0 = parallel_apply(
        func=applyPtSlOnT1,
        items=events.index,
        num_threads=numThreads,
        close=close,
        events=events,
        ptSl=ptSl_
    )

    # Step 5: Choose the first touched barrier
    events['t1'] = df0.dropna(how='all').min(axis=1)
    
    if side is None:
        events = events.drop('side', axis=1)
        
    return events


In [319]:
def get_vertical_barrier(t_events, close, delta=pd.Timedelta(minutes=60)):
    """
    For each event in t_events, find the timestamp in close.index 
    that is at least delta later. Returns a pd.Series of t1.
    """
    barrier_times = t_events + delta
    t1_idx = close.index.searchsorted(barrier_times)
    valid_idx = t1_idx[t1_idx < len(close)]
    t1 = pd.Series(close.index[valid_idx], index=t_events[:len(valid_idx)])
    return t1

In [320]:
t1 = get_vertical_barrier(t_events, df['close'], delta=pd.Timedelta(minutes=120))

In [321]:
t1

timestamp
2021-01-07 05:45:00   2021-01-07 07:45:00
2021-01-07 07:05:00   2021-01-07 09:05:00
2021-01-07 08:05:00   2021-01-07 10:05:00
2021-01-07 09:10:00   2021-01-07 11:10:00
2021-01-07 09:50:00   2021-01-07 11:50:00
                              ...        
2024-12-30 15:40:00   2024-12-30 17:40:00
2024-12-30 16:40:00   2024-12-30 18:40:00
2024-12-30 18:00:00   2024-12-30 20:00:00
2024-12-30 19:00:00   2024-12-30 21:00:00
2024-12-30 20:30:00   2024-12-30 22:30:00
Name: timestamp, Length: 24492, dtype: datetime64[ns]

In [322]:
trgt.describe()

count    24493.000000
mean         0.000640
std          0.000423
min          0.000062
25%          0.000377
50%          0.000542
75%          0.000770
max          0.004740
Name: NATR_12, dtype: float64

In [323]:
minTrgt = trgt.quantile(.3)
minTrgt
# minTrgt = 0

np.float64(0.0004106502722886354)

In [324]:
%%time

events = getEvents(
    close=df['close'],
    tEvents=t_events, 
    ptSl=[1,1], 
    trgt=trgt, 
    minTrgt=minTrgt, 
    numThreads=20, 
    t1=t1, 
    side=None
)

CPU times: user 4.24 s, sys: 277 ms, total: 4.52 s
Wall time: 4.29 s


In [325]:
events.head()

,t1,trgt
timestamp,,
2021-01-07 09:10:00,2021-01-07 09:45:00,0.000432
2021-01-07 09:50:00,2021-01-07 10:30:00,0.000422
2021-01-07 10:35:00,2021-01-07 12:00:00,0.000411
2021-01-07 12:00:00,2021-01-07 12:20:00,0.000446
2021-01-07 12:35:00,2021-01-07 13:10:00,0.000422


In [326]:
events = events.dropna().copy()

# Keep only intraday events

In [327]:
events = events[events.index.date==events.t1.dt.date]

## Create classification labels

In [328]:
def getBins(events, close, minRet=0.0, t1=None):
    '''
    Compute event's outcome (including side information, if provided).
    events is a DataFrame where:
    -events.index is event's starttime
    -events['t1'] is event's endtime
    -events['trgt'] is event's target
    -events['side'] (optional) implies the algo's position side
    -t1 is original vertical barrier series
    Case 1: ('side' not in events): bin in (-1,1) <-label by price action
    Case 2: ('side' in events): bin in (0,1) <-label by pnl (meta-labeling)
    '''
    # 1) prices aligned with events
    events_ = events.dropna(subset=['t1'])
    px = events_.index.union(events_['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    # 2) create out object
    out = pd.DataFrame(index=events_.index)
    out['ret'] = px.loc[events_['t1'].values].values / px.loc[
        events_.index] - 1
    if 'side' in events_: out['ret'] *= events_['side']  # meta-labeling
    out['bin'] = np.sign(out['ret'])

    if 'side' not in events_:
        # only applies when not meta-labeling.
        # to update bin to 0 when vertical barrier is touched, we need the
        # original vertical barrier series since the events['t1'] is the time
        # of first touch of any barrier and not the vertical barrier
        # specifically. The index of the intersection of the vertical barrier
        # values and the events['t1'] values indicate which bin labels needs
        # to be turned to 0.
        vtouch_first_idx = events[events['t1'].isin(t1.values)].index
        out.loc[vtouch_first_idx, 'bin'] = 0.

    if 'side' in events_: out.loc[out['ret'] <= 0, 'bin'] = 0  # meta-labeling
    out.loc[out['ret'].abs() < minRet, 'bin'] = 0. # 不做獲利過少的交易 (設為 0)
    # out = out.loc[out['ret'].abs() >= minRet] # 過濾掉獲利過少的交易 (刪掉)
    return out

In [329]:
labels = getBins(events, df['close'], minRet=0, t1=t1)

In [330]:
print(labels.ret.abs().describe())
print(labels.ret.abs().quantile(.3))

count    17012.000000
mean         0.001100
std          0.000802
min          0.000000
25%          0.000680
50%          0.000890
75%          0.001250
max          0.020367
Name: ret, dtype: float64
0.0007177626737453813


In [331]:
labels.bin.value_counts()

bin
 1.0    7040
-1.0    6791
 0.0    3181
Name: count, dtype: int64

In [332]:
labels["bin_class"] = labels["bin"] + 1

In [333]:
labeled_events = events.join(labels, how='inner')

In [334]:
labeled_events.head()

,t1,trgt,ret,bin,bin_class
timestamp,,,,,
2021-01-07 09:10:00,2021-01-07 09:45:00,0.000432,0.000435,1.0,2.0
2021-01-07 09:50:00,2021-01-07 10:30:00,0.000422,0.000531,1.0,2.0
2021-01-07 10:35:00,2021-01-07 12:00:00,0.000411,0.000690,1.0,2.0
2021-01-07 12:00:00,2021-01-07 12:20:00,0.000446,-0.000603,-1.0,0.0
2021-01-07 12:35:00,2021-01-07 13:10:00,0.000422,-0.000540,-1.0,0.0


## Save events to disk

In [335]:
labeled_events.to_pickle(DIRECTION_LABEL_FILE_PATH)

In [336]:
DIRECTION_LABEL_FILE_PATH

PosixPath('../data/labels/direction_labels/USDJPY-5m-20210101-20241231-CUSUM_0.001-TB.pkl')